Code for ingestion of the dataset

Code for exploration of the dataset

The code should be organized logically with appropriate comments

Best practice: someone who is completely unfamiliar with your project and code can easily understand what you have done and run the code error-free. 

The code must run error-free.

Clearly mark where the week starts and ends.

# Need to add pip install yfinance for prof

In [1]:
#Go to repo
%cd /home/jupyter-guotq/HelpHerInvest

/home/jupyter-guotq/HelpHerInvest


In [2]:
#Sync latest from GitHub before editing
!git pull --rebase origin main

From https://github.com/tongyuguo/HelpHerInvest
 * branch            main       -> FETCH_HEAD
Already up to date.


In [3]:
# libraries

import time
import requests
import pandas as pd
#%pip install yfinance --quiet
import yfinance as yf
from pathlib import Path
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Code for ingestion of the dataset

'''
# Locate repo root and Data folder

def find_repo_root(start_path=None):
    """Walk up directories until .git is found"""
    p = Path(start_path or Path.cwd()).resolve()
    for parent in [p] + list(p.parents):
        if (parent / ".git").exists():
            return parent
    # fallback to current working directory
    return p

REPO_ROOT = find_repo_root()
DATA_DIR = REPO_ROOT / "Data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Output file (csv inside a zip)
output_file = "stock_symbols_new_small.csv.zip"
output_path = DATA_DIR / output_file
'''

repo_root = Path("/home/jupyter-guotq/HelpHerInvest")

output_path = repo_root / "Data" / "testing_small.csv.zip"

output_path.parent.mkdir(parents=True, exist_ok=True)

output_file = "testing_small.csv.zip"

# SEC Parameters

SEC_URL = "https://www.sec.gov/files/company_tickers_exchange.json"
SEC_HEADERS = {"User-Agent": "YourAppName your_email@example.com"}  # required by SEC

def get_universe_from_sec(limit):
    r = requests.get(SEC_URL, headers=SEC_HEADERS, timeout=30)
    r.raise_for_status()
    j = r.json()
    df = pd.DataFrame(j["data"], columns=j["fields"])
    df = df.rename(columns={"ticker": "symbol", "name": "company_name"})
    df["symbol"] = df["symbol"].str.upper()
    return df[["symbol", "company_name"]].drop_duplicates().head(limit)

def yf_fetch_info(symbol: str) -> dict:
    candidates = [symbol, symbol.replace("-", ".")]
    for sym in candidates:
        try:
            t = yf.Ticker(sym)
            info = t.get_info()
            if info and isinstance(info, dict) and info.get("quoteType") in ("EQUITY", "ETF"):
                return info
        except Exception:
            pass
    return {}

def build_base_table(limit, sleep_s=0.35):
    universe = get_universe_from_sec(limit=limit)
    print("Symbols pulled:", len(universe.index))
    rows = []
    count = 0
    for sym in universe["symbol"].tolist():
        rows.append(yf_fetch_info(sym))
        count += 1
        time.sleep(sleep_s)
        if count % 200 == 0:
            print(f"{count} rows completed")

    facts = pd.DataFrame(rows)

    df_cols = list(facts.columns)
    added = ["symbol", "company_name"]
    cols = df_cols + added

    base = (
        universe
        .merge(facts, on="symbol", how="left")
        [cols]
        .drop_duplicates(subset=["symbol"])
    )
    return base


# Run + Save

df_base = build_base_table(limit=200, sleep_s=0.35)

print(df_base.head(10))
print(df_base.shape)
print(df_base.columns)

# Save as CSV inside a zip
df_base.to_csv(output_path, index=False, compression="zip")
print(f"Saved: {output_path}")

'''
# Load later (same folder)
df = pd.read_csv(output_path, compression="zip")
grouped = df.groupby("sector")["sector"].count()
print(grouped)
'''

Symbols pulled: 200
200 rows completed
                    address1           city state         zip        country  \
0  2788 San Tomas Expressway    Santa Clara    CA       95051  United States   
1  1600 Amphitheatre Parkway  Mountain View    CA       94043  United States   
2         One Apple Park Way      Cupertino    CA       95014  United States   
3          One Microsoft Way        Redmond    WA  98052-6399  United States   
4     410 Terry Avenue North        Seattle    WA  98109-5210  United States   
5                 1 Meta Way     Menlo Park    CA       94025  United States   
6          3421 Hillview Ave      Palo Alto    CA       94304  United States   
7               1 Tesla Road         Austin    TX       78725  United States   
8         3555 Farnam Street          Omaha    NE       68131  United States   
9           1 Customer Drive    Bentonville    AR       72716  United States   

            phone                            website  \
0    408 486 2000       

'\n# Load later (same folder)\ndf = pd.read_csv(output_path, compression="zip")\ngrouped = df.groupby("sector")["sector"].count()\nprint(grouped)\n'

In [5]:
# Dependent Variables

## Reading Symbols from Dataset ##
#symbols = pd.read_csv("testing_small.csv.zip")
symbols = df_base
#tickers = symbols['symbol'].tolist()
#tickers = symbols['symbol'].values.tolist() 
tickers = symbols['symbol'].iloc[:, 0].tolist()
print(tickers[:20])      # first 20
#tickers.remove("SPY")
tickers.remove('AAPL')
tickers = tickers[:50]  # limit to 50 for testing
#benchmark = 'SPY'
benchmark = 'AAPL'
all_tickers = tickers + [benchmark]

## Download Price Data ##
prices = yf.download(
    all_tickers,
    start='2010-01-01',
    auto_adjust=False,
    progress=False
)['Adj Close']

prices = prices.dropna(how='all')

print(prices.head(10))

## Compute Features ##
monthly_px = prices.resample('ME').last()  # month-end prices
mom_1m  = monthly_px / monthly_px.shift(1)  - 1  # 1-month momentum
mom_3m  = monthly_px / monthly_px.shift(3)  - 1  # 3-month momentum
mom_6m  = monthly_px / monthly_px.shift(6)  - 1  # 6-month momentum
mom_12m = monthly_px / monthly_px.shift(12) - 1  # 12-month momentum
mom_12m_ex_1m = (monthly_px.shift(1) / monthly_px.shift(12)) - 1  # 12-month momentum excluding most recent month

rel_3m_spy  = mom_3m.sub(mom_3m[benchmark], axis=0)  # relative strength against S&P 3-month
rel_6m_spy  = mom_6m.sub(mom_6m[benchmark], axis=0)  # relative strength against S&P 6-month
rel_12m_spy = mom_12m.sub(mom_12m[benchmark], axis=0) # relative strength against S&P 12-month

daily_ret = prices.pct_change() # daily returns

vol_3m = (daily_ret.rolling(63).std() * np.sqrt(252)).resample("M").last() # 3-month volatility
vol_6m = (daily_ret.rolling(126).std() * np.sqrt(252)).resample("M").last() # 6-month volatility


roll_max_6m  = monthly_px.rolling(6).max()  # 6-month rolling max
roll_max_12m = monthly_px.rolling(12).max() # 12-month rolling max

drawdown_6m  = monthly_px / roll_max_6m  - 1  # 6-month drawdown
drawdown_12m = monthly_px / roll_max_12m - 1  # 12-month drawdown

dma_200 = prices.rolling(200).mean().resample("M").last() # 200-day moving average
pct_above_200dma = monthly_px / dma_200 - 1  # pct above 200-day moving average


## Combine Features ##
X = pd.concat(
    {
        "mom_1m": mom_1m[tickers],
        "mom_3m": mom_3m[tickers],
        "mom_6m": mom_6m[tickers],
        "mom_12m": mom_12m[tickers],
        "mom_12m_ex_1m": mom_12m_ex_1m[tickers],
        "rel_3m_spy": rel_3m_spy[tickers],
        "rel_6m_spy": rel_6m_spy[tickers],
        "rel_12m_spy": rel_12m_spy[tickers],
        "vol_3m": vol_3m[tickers],
        "vol_6m": vol_6m[tickers],
        #"downside_vol_6m": downside_vol_6m[tickers],
        "drawdown_6m": drawdown_6m[tickers],
        "drawdown_12m": drawdown_12m[tickers],
        "pct_above_200dma": pct_above_200dma[tickers],
    },
    axis=1
)


## Standardize Data Function - z score ##
def zscore_cs(row: pd.Series) -> pd.Series:
    # row contains values across tickers for a single feature at a single date
    mu = row.mean()
    sd = row.std(ddof=0)
    if sd == 0 or np.isnan(sd):
        return row * 0.0
    return (row - mu) / sd # calcs z-score


## Normalize per feature across tickers at each date ##
X_z = X.copy()
print("X_z before normalization:")
print(X_z.head(20))

for feat in X.columns.get_level_values(0).unique():
    X_z[feat] = X[feat].apply(zscore_cs, axis=1)

print("X_z after normalization:")
print(X_z.head(20))

## Flatten X_z table so each  ticker is a row ##
X_panel = (
    X_z.stack(level=1)              # index becomes (Date, Ticker)
      .rename_axis(index=["Date","Ticker"])
      .reset_index()
)

print(X_panel.head())
print("-----  -----  -----")
print(X_panel.tail())
print("-----  -----  -----")

print("Size of dataset:",
"Rows:",X_panel.shape[0],
"Columns:",X_panel.shape[1])  



#X_panel.to_csv("dependent_variables_technicals.csv", index=False)

['NVDA', 'GOOGL', 'AAPL', 'MSFT', 'AMZN', 'META', 'AVGO', 'TSLA', 'BRK-B', 'WMT', 'LLY', 'JPM', 'SPY', 'V', 'XOM', 'ASML', 'JNJ', 'ORCL', 'MA', 'MU']
Ticker          AAPL  ABBV       AMAT   AMD    AMZN       ASML      AVGO  \
Date                                                                       
2010-01-04  6.418385   NaN  11.027140  9.70  6.6950  32.425720  1.328563   
2010-01-05  6.429479   NaN  10.942317  9.71  6.7345  32.678329  1.338425   
2010-01-06  6.327211   NaN  10.919181  9.57  6.6125  32.977695  1.348991   
2010-01-07  6.315515   NaN  10.803513  9.47  6.5000  32.060867  1.340538   
2010-01-08  6.357502   NaN  11.219923  9.43  6.6760  31.293726  1.350401   
2010-01-11  6.301419   NaN  11.466683  9.14  6.5155  30.629503  1.358854   
2010-01-12  6.229738   NaN  10.950027  8.65  6.3675  30.676268  1.338425   
2010-01-13  6.317613   NaN  11.004006  9.15  6.4555  31.527617  1.297568   
2010-01-14  6.281025   NaN  11.065697  9.00  6.3675  31.312429  1.298272   
2010-01-15  6.

In [6]:
# Independent Variable


#rand_symbols = symbols['symbol'].head(50).tolist()
rand_symbols = symbols['symbol'].iloc[:, 0].head(50).tolist()
print(rand_symbols)

def forward_excess_return_monthly(tickers, benchmark="SPY", start="2010-01-01", end=None, horizon_months=3):

    universe = list(dict.fromkeys(list(tickers) + [benchmark]))

    px_daily = yf.download(
        universe, start=start, end=end, auto_adjust=False, progress=False
    )["Adj Close"]

    px_m = px_daily.resample("M").last().dropna(subset=[benchmark])

    fwd_ret = px_m.shift(-horizon_months) / px_m - 1.0
    bench_fwd = fwd_ret[benchmark].rename("bench_fwd_return")

    # Wide to long in one go
    out = pd.DataFrame(index=px_m.index)
    out["bench_fwd_return"] = bench_fwd

    for t in tickers:
        out[f"adj_close_{t}"] = px_m[t]
        out[f"fwd_return_{t}"] = fwd_ret[t]
        out[f"fwd_excess_{t}"] = fwd_ret[t] - bench_fwd

    return out

df_y_m = forward_excess_return_monthly(rand_symbols, benchmark="SPY", start="2010-01-01", horizon_months=6)
print(df_y_m.head(10))
print(df_y_m.tail(10))

bench_df = (
    df_y_m[["bench_fwd_return"]]
    .rename(columns={"bench_fwd_return": "bench_fwd_return"})
    .reset_index()
)

# Keep only stock-level columns
stock_cols = [c for c in df_y_m.columns if "_" in c and not c.startswith("fwd_ret_bench")]

y_long = (
    df_y_m[stock_cols]
    .reset_index()
    .melt(id_vars="Date", var_name="metric_ticker", value_name="value")
)

# Split "adj_close_AAPL" -> metric="adj_close", ticker="AAPL"
y_long[["metric", "ticker"]] = y_long["metric_ticker"].str.rsplit("_", n=1, expand=True)

y_long = y_long.drop(columns="metric_ticker")

df_y_m_output = (
    y_long
    .pivot(index=["Date", "ticker"], columns="metric", values="value")
    .reset_index()
)

df_y_m_output = df_y_m_output[["Date", "ticker", "adj_close", "fwd_excess", "fwd_return"]]
df_y_m_output = df_y_m_output[df_y_m_output["ticker"] != "return"]

print(df_y_m_output.head(10))
print(df_y_m_output.tail(10))


# Save to CSV
#df_y_m_output.to_csv("y_variable_forward_excess_return.csv", index=False)

['NVDA', 'GOOGL', 'AAPL', 'MSFT', 'AMZN', 'META', 'AVGO', 'TSLA', 'BRK-B', 'WMT', 'LLY', 'JPM', 'SPY', 'V', 'XOM', 'ASML', 'JNJ', 'ORCL', 'MA', 'MU', 'COST', 'BABA', 'AMD', 'ABBV', 'PLTR', 'BAC', 'HD', 'NFLX', 'PG', 'CVX', 'KO', 'GE', 'CSCO', 'HSBC', 'LRCX', 'CAT', 'AZN', 'TM', 'NVS', 'MS', 'GS', 'NVO', 'WFC', 'PM', 'IBM', 'CYATY', 'SAP', 'RTX', 'MRK', 'AMAT']
            bench_fwd_return  adj_close_NVDA  fwd_return_NVDA  \
Date                                                            
2010-01-31          0.035951        0.352752        -0.402859   
2010-02-28         -0.040575        0.371318        -0.424074   
2010-03-31         -0.014642        0.398823        -0.328735   
2010-04-30          0.007416        0.360087        -0.234882   
2010-05-31          0.094369        0.301180         0.035769   
2010-06-30          0.231235        0.234022         0.508326   
2010-07-31          0.179372        0.210643         1.602830   
2010-08-31          0.277816        0.213852        

In [7]:
# Check what changed
!git status -sb
# Stage + commit
!git add -A
!git commit -m "hopefull final testing of working and output directory"
# Push
!git push

## main...origin/main
?? Data/testing_small.csv.zip
[main f665758] hopefull final testing of working and output directory
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Data/testing_small.csv.zip
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 296.20 KiB | 21.16 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/tongyuguo/HelpHerInvest.git
   b4ec62c..f665758  main -> main
